# KubeVela Demo - Product Catalog Service

This notebook demonstrates KubeVela's unified application delivery model with:
- Single YAML file for app + infrastructure + workflow + tests
- Built-in progressive delivery (dev → staging → prod)
- Automated functional API testing
- Policy-based environment configuration

**Total: 258 lines in 1 file**

## Prerequisites Check

In [ ]:
# Verify cluster and tools
!kubectl get nodes
!vela version
!k3d registry list

## Step 1: Build and Push Application Image

Build the Python Flask application and push to local k3d registry.

In [ ]:
%%bash
cd app
echo "Building Docker image..."
DOCKER_BUILDKIT=0 docker build -t kv-product-cat-api:v1.0.0 .

echo "Tagging for local registry..."
docker tag kv-product-cat-api:v1.0.0 localhost:5000/kv-product-cat-api:v1.0.0

echo "Pushing to k3d registry..."
docker push localhost:5000/kv-product-cat-api:v1.0.0

echo "✓ Image available at k3d-registry.localhost:5000/kv-product-cat-api:v1.0.0"

## Step 2: Verify AWS Credentials

Ensure AWS credentials are configured for S3 access.

In [ ]:
%%bash
# Check if credentials secret exists
kubectl get secret aws-credentials -n default 2>/dev/null || echo "⚠️  AWS credentials secret not found"

# Test AWS connectivity
set -a && source ../.env.aws && set +a && aws sts get-caller-identity

## Step 3: Review the Application Definition

The single `application.yaml` file contains:
- **Components**: Webservice + S3 bucket
- **Traits**: HPA, Security Context, Resource limits
- **Policies**: Topology and environment-specific overrides
- **Workflow**: Progressive delivery with functional tests

In [ ]:
# Display the application definition
!cat kubevela/application.yaml | head -80

## Step 4: Deploy Application

Deploy the application - this will:
1. Create S3 bucket via Crossplane
2. Deploy to dev environment
3. Run functional tests (POST + GET)
4. Wait for approval to proceed to staging

In [ ]:
%%bash
cd kubevela
vela up -f application.yaml

## Step 5: Check Application Status

In [ ]:
!vela status kv-product-catalog

## Step 6: Verify Dev Environment

Check that resources are deployed in the dev namespace.

In [ ]:
!kubectl get pods,svc,hpa -n dev

## Step 7: Check S3 Bucket Creation

In [ ]:
!set -a && source ../.env.aws && set +a && aws s3 ls | grep tenant-atlantis-kv-prodcat-images

## Step 8: Promote to Staging

Resume the workflow to deploy to staging environment.

In [ ]:
!vela workflow resume kv-product-catalog

In [ ]:
# Wait for staging deployment and check status
import time
time.sleep(30)
!kubectl get pods,hpa -n staging

## Step 11: Promote to Production

Production promotion happens automatically after 1m

In [ ]:
!vela workflow resume kv-product-catalog

In [ ]:
# Wait for production deployment and check status
import time
time.sleep(60)
!kubectl get pods,hpa -n prod

## Step 12: View Complete Deployment

Show resources across all environments.

In [ ]:
print("=== DEV ENVIRONMENT ===")
!kubectl get pods,hpa -n dev

print("\n=== STAGING ENVIRONMENT ===")
!kubectl get pods,hpa -n staging

print("\n=== PROD ENVIRONMENT ===")
!kubectl get pods,hpa -n prod

## Key Takeaways

### KubeVela Benefits:
- ✅ **258 lines** in a single YAML file
- ✅ **65% less code** vs traditional approach (746 lines)
- ✅ **83% fewer files** (1 vs 6)
- ✅ Built-in progressive delivery with approval gates
- ✅ Automated functional API testing (POST + GET)
- ✅ Policy-based environment configuration
- ✅ Infrastructure as application components
- ✅ No external CI/CD required
- ✅ Declarative YAML vs imperative Go code

### What Happened:
1. Single `vela up` command deployed everything
2. S3 bucket automatically created via Crossplane
3. Application deployed to dev with HPA, security, and resources
4. Functional tests validated API (POST + GET)
5. Manual approval gates between environments
6. Same workflow promoted to staging and prod
7. Environment-specific policies applied (different HPA settings)

## Cleanup

In [ ]:
%%bash
# Delete the application (Crossplane automatically cleans up S3)
vela delete kv-product-catalog

# Delete namespaces
kubectl delete namespace dev staging prod